# 7-round Speck (FNN)


## Imports

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pipeline import *

2023-05-22 15:45:43.085476: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from dataset_rr.make_train_data import make_train_data
from dataset_rr.speck import Speck

## Importing the dataset

In [4]:
n_train_samples = 10**6
n_eval_samples = 10**5
n_rounds = 7

cipher = Speck(n_rounds=n_rounds)

key = cipher.draw_keys(1)

In [5]:
train_samples, train_labels = make_train_data(n_train_samples, cipher, key)
test_samples, test_labels = make_train_data(n_eval_samples, cipher, key)

In [6]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (1000000, 32)
===== Label Shape: (32,)
===== Training Samples Shape: (1000000, 32)
===== Sample Shape: (32,)
===== Testing Labels Shape: (100000, 32)
===== Testing Samples Shape: (100000, 32)


## Creating the model

In [7]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [19]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*16

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    decay_rate=0.01)
learning_rate = 0.1

optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 100
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [20]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               16896     
                                                                 
 dense_7 (Dense)             (None, 512)               262656    
                                                                 
 dense_8 (Dense)             (None, 512)               262656    
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dense_11 (Dense)            (None, 32)                16416     
                                                                 
Total params: 1,083,936
Trainable params: 1,083,936
No

### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [22]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/100
180/180 [==============================] - 28s 156ms/step - loss: 0.2500 - accuracy: 0.0328 - binary_accuracy: 0.5003 - val_loss: 0.2500 - val_accuracy: 3.1000e-04 - val_binary_accuracy: 0.4994
Epoch 2/100
180/180 [==============================] - 37s 207ms/step - loss: 0.2500 - accuracy: 0.0207 - binary_accuracy: 0.5004 - val_loss: 0.2500 - val_accuracy: 0.0016 - val_binary_accuracy: 0.4996
Epoch 3/100
180/180 [==============================] - 32s 178ms/step - loss: 0.2500 - accuracy: 0.0301 - binary_accuracy: 0.5007 - val_loss: 0.2500 - val_accuracy: 0.0846 - val_binary_accuracy: 0.4999
Epoch 4/100
180/180 [==============================] - 34s 191ms/step - loss: 0.2500 - accuracy: 0.0299 - binary_accuracy: 0.5010 - val_loss: 0.2500 - val_accuracy: 0.0013 - val_binary_accuracy: 0.4999
Epoch 5/100
180/180 [==============================] - 31s 173ms/step - loss: 0.2500 - accuracy: 0.0354 - binary_accuracy: 0.5014 - val_loss: 0.2500 - val_accuracy: 0.0510 - val_binary_acc

In [23]:
h_loss = history.history['loss']
h_val_loss = history.history['val_loss']
h_val_accuracy = history.history['val_accuracy']

print(h_loss)
print(h_val_loss)
print(h_val_accuracy)

[0.2500031292438507, 0.2500009834766388, 0.2500002682209015, 0.24999885261058807, 0.24999623000621796, 0.2499919980764389, 0.24998492002487183, 0.24997209012508392, 0.24995243549346924, 0.24991726875305176, 0.24985623359680176, 0.24976029992103577, 0.24960914254188538, 0.24937887489795685, 0.2490483671426773, 0.24862118065357208, 0.24807874858379364, 0.24744610488414764, 0.24673549830913544, 0.2459687441587448, 0.24518337845802307, 0.24437923729419708, 0.24356478452682495, 0.24277286231517792, 0.2420131415128708, 0.24127574265003204, 0.24058039486408234, 0.2399139553308487, 0.23929142951965332, 0.23869112133979797, 0.23812559247016907, 0.2375909835100174, 0.23709145188331604, 0.23661784827709198, 0.23617562651634216, 0.2357586771249771, 0.23536498844623566, 0.23498433828353882, 0.23463161289691925, 0.2342890053987503, 0.23395724594593048, 0.23366059362888336, 0.2333681434392929, 0.23310396075248718, 0.2328583151102066, 0.2326032966375351, 0.23236645758152008, 0.23214615881443024, 0.231

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [24]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

20/20 [==============================] - 1s 51ms/step - loss: 0.2747 - accuracy: 0.0327 - binary_accuracy: 0.4998
Test loss: 0.274662047624588
Test accuracy: 0.03271999955177307


In [25]:
pred = neural_network.predict(test_samples[:10])

1/1 [==============================] - 0s 78ms/step


In [26]:
pred[0]

array([0.4525163 , 0.17726934, 0.69340664, 0.4365242 , 0.76196057,
       0.4180476 , 0.53752935, 0.59868187, 0.35439357, 0.6448451 ,
       0.5328131 , 0.24068972, 0.14234057, 0.79492795, 0.4710139 ,
       0.44469225, 0.28196803, 0.42624196, 0.5659851 , 0.61326224,
       0.8048344 , 0.08102047, 0.36428064, 0.7318592 , 0.6697646 ,
       0.57372767, 0.24984804, 0.2781828 , 0.39109993, 0.4496559 ,
       0.7173713 , 0.24956965], dtype=float32)

In [27]:
pred[1]

array([0.37754977, 0.5799621 , 0.4138214 , 0.42336604, 0.20023662,
       0.6535071 , 0.48306206, 0.38928786, 0.48633155, 0.5324746 ,
       0.6377058 , 0.6476289 , 0.5499287 , 0.5077108 , 0.48496833,
       0.57353663, 0.37935862, 0.50139517, 0.35069612, 0.3383927 ,
       0.54697484, 0.61946917, 0.35271114, 0.6439532 , 0.5327635 ,
       0.7996704 , 0.5937966 , 0.6946059 , 0.46007985, 0.43709946,
       0.15768008, 0.64882374], dtype=float32)